In [1]:
import rioxarray
import rasterio
import pandas as pd
import os
import geopandas
import re
import zipfile
import pickle
import shutil
from datetime import datetime
from tqdm import tqdm

In [2]:
data_input_dir = "data/gpm-imerg"
aoi_dir = "data/aoi.geojson"
geodf = geopandas.read_file(aoi_dir)
grace_dates_dir = "data/gpm_dates.pickle"
#imerg data directory
gpm_imerg_dir = "/Users/pawel/jsimpsonhttps.pps.eosdis.nasa.gov/imerg/gis" 

In [3]:
geodf.bounds

,minx,miny,maxx,maxy
0,18.406925,50.476079,19.680133,51.179343


In [4]:
with open(grace_dates_dir, 'rb') as f:
    grace_dates = pickle.load(f)

In [5]:
grace_dates = [datetime.strptime(date, "%Y-%m-%d") for date in grace_dates]

In [6]:
def generate_date_path(date):
    return os.path.join(gpm_imerg_dir, date.strftime('%Y'), date.strftime('%m'))

In [7]:
def handle_file(date):
    file_path = generate_date_path(date)
    extracted_path = os.path.join(file_path, "extracted")
    zips = os.listdir(file_path)
    file_pattern = ".*" + date.strftime('%Y%m%d') + "-.*E235959.*7day.zip"
    file_patter = re.compile(file_pattern)
    zips = list(filter(file_patter.match, zips))
    zip = zips[0]
    zip_path = os.path.join(file_path, zip)
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_path)
    except:
        print("Error in: " + zip_path)

    files = os.listdir(extracted_path)
    file_pattern = ".*7day.tif"
    file_patter = re.compile(file_pattern)
    file_list = list(filter(file_patter.match, files))
    file = file_list[0]
    tif_path = os.path.join(extracted_path, file)
    data = rasterio.open(tif_path)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data = data.rio.write_crs("EPSG:4326")
    masked = data.to_masked_array()
    shutil.rmtree(extracted_path)
    return masked.data

In [8]:
def aggregate_imerg_data(grace_date):
    file = handle_file(grace_date)
    return file


In [9]:
df = pd.DataFrame(columns = ["date", "value"])
for grace_date in tqdm(grace_dates):
    df = df.append({"date": grace_date.strftime("%Y-%m-%d"), "value": aggregate_imerg_data(grace_date)}, ignore_index=True)

  0%|          | 0/1021 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/pawel/jsimpsonhttps.pps.eosdis.nasa.gov/imerg/gis/2002/04'

In [ ]:
df

,date,value
0,2002-04-08,"[[nan, nan, nan, nan, nan, nan, 95.0, 79.0, 76..."
1,2002-04-15,"[[nan, nan, nan, nan, nan, nan, 142.0, 121.0, ..."
2,2002-04-22,"[[nan, nan, nan, nan, nan, nan, 104.0, 72.0, 7..."
3,2002-04-29,"[[nan, nan, nan, nan, nan, nan, 187.0, 163.0, ..."
4,2002-05-06,"[[nan, nan, nan, nan, nan, nan, 406.0, 319.0, ..."
...,...,...
1016,2021-09-27,"[[nan, nan, nan, nan, nan, nan, 269.0, 258.0, ..."
1017,2021-10-04,"[[nan, nan, nan, nan, nan, nan, 228.0, 229.0, ..."
1018,2021-10-11,"[[nan, nan, nan, nan, nan, nan, 327.0, 356.0, ..."
1019,2021-10-18,"[[nan, nan, nan, nan, nan, nan, 259.0, 281.0, ..."


In [ ]:
gpm_imerg_zip_dir = "data/gpm-imerg/zip"
gpm_imerg_extracted_dir = "data/gpm-imerg/extracted"
zips = os.listdir(gpm_imerg_zip_dir)
file_pattern = ".*.zip"
file_patter = re.compile(file_pattern)
zips = list(filter(file_patter.match, zips))
for zipf in zips:
    try:
        with zipfile.ZipFile(os.path.join(gpm_imerg_zip_dir, zipf), 'r') as zip_ref:
            zip_ref.extractall(gpm_imerg_extracted_dir)
    except:
        print("Error in: " + zipf)
        

Error in: 3B-MO-L.GIS.IMERG.20190601.V06B.zip


In [ ]:
gpm_imerg_extracted_dir = "data/gpm-imerg/extracted"
files = os.listdir(gpm_imerg_extracted_dir)
file_pattern = ".*V06B.tif"
file_patter = re.compile(file_pattern)
file_list = list(filter(file_patter.match, files))

In [ ]:
df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(gpm_imerg_extracted_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data = data.rio.write_crs("EPSG:4326")
    masked = data.to_masked_array()
    datetime = f[18:22] + "-" + f[22:24] + "-" + f[24:26]
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
    #df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)

/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1947/1244022856.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1947/1244022856.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1947/1244022856.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_1947/1244022856.py:13: FutureWarning: The frame.append method 

In [ ]:
df.to_pickle("data/gpm-imerg_weekly_df.pickle")

In [ ]:
df

,date,value
0,2002-04-08,"[[nan, nan, nan, nan, nan, nan, 95.0, 79.0, 76..."
1,2002-04-15,"[[nan, nan, nan, nan, nan, nan, 142.0, 121.0, ..."
2,2002-04-22,"[[nan, nan, nan, nan, nan, nan, 104.0, 72.0, 7..."
3,2002-04-29,"[[nan, nan, nan, nan, nan, nan, 187.0, 163.0, ..."
4,2002-05-06,"[[nan, nan, nan, nan, nan, nan, 406.0, 319.0, ..."
...,...,...
1016,2021-09-27,"[[nan, nan, nan, nan, nan, nan, 269.0, 258.0, ..."
1017,2021-10-04,"[[nan, nan, nan, nan, nan, nan, 228.0, 229.0, ..."
1018,2021-10-11,"[[nan, nan, nan, nan, nan, nan, 327.0, 356.0, ..."
1019,2021-10-18,"[[nan, nan, nan, nan, nan, nan, 259.0, 281.0, ..."
